In [1]:
import requests
from bs4 import BeautifulSoup
from dateutil import parser
import pandas as pd
import datetime
import re

In [2]:
def search_page(vol, page):
    print(f'Scraping from volumn {vol} page {page}', end = ":\t")
    url = f"https://nejm.jp/abstract/vol{vol}.p{page}"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content)
    results = [parag.text for parag in soup.select(".add")]
    try:
        new = soup.select("#sectionWrap a")[0].attrs['href']
    except Exception:
        print("0 training sets added.")
        return []
    en_page = requests.get(new, headers = headers)
    en_soup = BeautifulSoup(en_page.content)
    en_results = [parag.text for parag in en_soup.select("#article_Abstract .f-body")]
    if not en_results:
        en_results = [parag.text for parag in en_soup.select("#article_body .f-body")]
    tmp = [(url, jp, new, en) for jp, en in zip(results, en_results)]
    print(f"{len(tmp)} training sets added.")
    return tmp

def search_volumn(vol, num):
    url = f"https://nejm.jp/contents/idx.vol{vol}.no{num}"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
    page = requests.get(url, headers = headers)
    soup = BeautifulSoup(page.content)
    results = [int(re.match('/abstract/vol\d+.p(\d+)', parag.attrs['href'])[1]) for parag in soup.select(".link02 a")]
    if not soup.select(".pageNum"):
        raise Exception("No items found.")
    return sum([search_page(vol, page) for page in results], [])

def search_all(vol_start = 336, vol_end = None):
    vol, num = vol_start, 1
    total_results = []
    vol_results = []
    while not vol_end or vol <= vol_end:
        try:
            vol_results += search_volumn(vol, num)
            num += 1
        except Exception as e:
            if num == 1:
                break
            if vol_results:
                pd.DataFrame(vol_results, columns = ['jp_url', 'jp_txt', 'en_url', 'en_txt']).to_csv(f'jp_en/vol_{vol}.csv', index = False)
                total_results += vol_results
                vol_results = []
            vol, num = vol + 1, 1
    return total_results

In [5]:
JP=search_all(383)
JP

Scraping from volumn 383 page 13:	4 training sets added.
Scraping from volumn 383 page 24:	4 training sets added.
Scraping from volumn 383 page 35:	4 training sets added.
Scraping from volumn 383 page 49:	4 training sets added.
Scraping from volumn 383 page 109:	4 training sets added.
Scraping from volumn 383 page 120:	4 training sets added.
Scraping from volumn 383 page 129:	4 training sets added.
Scraping from volumn 383 page 141:	4 training sets added.
Scraping from volumn 383 page 207:	4 training sets added.
Scraping from volumn 383 page 218:	4 training sets added.
Scraping from volumn 383 page 229:	4 training sets added.
Scraping from volumn 383 page 240:	4 training sets added.
Scraping from volumn 383 page 309:	4 training sets added.
Scraping from volumn 383 page 321:	4 training sets added.
Scraping from volumn 383 page 334:	4 training sets added.
Scraping from volumn 383 page 347:	4 training sets added.
Scraping from volumn 383 page 359:	4 training sets added.


[('https://nejm.jp/abstract/vol383.p13',
  'アフリカの資源の少ない環境での，肺炎の小児に対する抗菌薬の適切な治療期間に関するエビデンスは不足している．',
  'https://www.nejm.org/doi/full/10.1056/NEJMoa1912400',
  'Evidence regarding the appropriate duration of treatment with antibiotic agents in children with pneumonia in low-resource settings in Africa is lacking.'),
 ('https://nejm.jp/abstract/vol383.p13',
  'マラウイのリロングウェで，胸部陥凹を伴う肺炎（持続期間が 14 日間未満の咳嗽，または呼吸困難に，明らかな胸壁の陥凹を伴う；年齢に比した多呼吸の有無は問わない）の小児に対するアモキシシリンの 3 日間の投与は，5 日間の投与よりも有効性が低いかどうかを明らかにするため，二重盲検無作為化対照非劣性試験を行った．胸部陥凹を伴う肺炎を有する生後 2～59 ヵ月のヒト免疫不全ウイルス（HIV）陰性の小児を，アモキシシリンを 1 日 2 回，3 日間投与する群と 5 日間投与する群に無作為に割り付けた．児は 14 日間追跡した．主要転帰は 6 日目までの治療失敗とした．3 日間レジメンの 5 日間レジメンに対する非劣性は，3 日間群の治療失敗例の割合が 5 日間群の 1.5 倍を超えない場合に示されるものとした．事前に規定した副次的解析には，14 日目までの治療失敗または再発の評価を含めた．',
  'https://www.nejm.org/doi/full/10.1056/NEJMoa1912400',
  'We conducted a double-blind, randomized, controlled, noninferiority trial in Lilongwe, Malawi, to determine whether treatment with amoxicillin for 3 days is less effective than tre

In [6]:
pd.DataFrame(JP, columns = ['jp_url', 'jp_txt', 'en_url', 'en_txt']).to_csv(f'jp_en/total2.csv', index = False)

In [163]:
data = pd.DataFrame(i, columns = ['jp_url', 'jp_txt', 'en_url', 'en_txt'])
data.head()

NameError: name 'i' is not defined